# 第9章: 基本的なデータ分析パイプラインの構築

このノートブックでは、NYC Taxiデータを使用して、Apache Icebergを中心としたデータ分析パイプラインを構築します。

## 1. 環境のセットアップ

In [ ]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, hour, dayofweek, month, year, date_format, to_date
from datetime import datetime
import pandas as pd

CATALOG = "demo"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

In [ ]:
# SparkSessionの作成
spark = (
    SparkSession.builder
        .appName("NYC Taxi Data Pipeline")
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", CATALOG)
        .getOrCreate()
)

In [ ]:
%sql spark

In [ ]:
# カタログとデータベースの作成
spark.sql("CREATE DATABASE IF NOT EXISTS nyc_taxi")

## 2. NYC Taxiデータの取得と初期探索

In [ ]:
# NYC Taxiデータの読み込み（既にDockerイメージ内に配置済み）
df_taxi = spark.read.parquet("/home/jovyan/data/yellow_tripdata_2022-01.parquet")

### データの概要を確認

In [ ]:
print(f"レコード数: {df_taxi.count():,}")
print(f"カラム数: {len(df_taxi.columns)}")
df_taxi.printSchema()

In [ ]:
df_taxi.show(5, truncate=False)

In [ ]:
df_taxi.select("trip_distance", "fare_amount", "tip_amount", "total_amount") \
    .summary("count", "mean", "stddev", "min", "max") \
    .show()

## 3. Icebergテーブルへのデータ保存

In [ ]:
%%sql
CREATE OR REPLACE TABLE demo.nyc_taxi.yellow_trips_raw (
    VendorID BIGINT,
    tpep_pickup_datetime TIMESTAMP_NTZ,
    tpep_dropoff_datetime TIMESTAMP_NTZ,
    passenger_count DOUBLE,
    trip_distance DOUBLE,
    RatecodeID DOUBLE,
    store_and_fwd_flag STRING,
    PULocationID BIGINT,
    DOLocationID BIGINT,
    payment_type BIGINT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    total_amount DOUBLE,
    congestion_surcharge DOUBLE,
    airport_fee DOUBLE
)
USING iceberg
PARTITIONED BY (month(tpep_pickup_datetime))

In [ ]:
%%sql
DESCRIBE TABLE demo.nyc_taxi.yellow_trips_raw

In [ ]:
df_taxi.writeTo("demo.nyc_taxi.yellow_trips_raw").append()

In [ ]:
%%sql
SELECT * FROM demo.nyc_taxi.yellow_trips_raw limit 10;

## 4. データ変換と集計テーブルの作成

### 時間別集計テーブル（hourly_stats）

In [ ]:
%%sql
CREATE OR REPLACE TABLE demo.nyc_taxi.hourly_stats
USING iceberg
PARTITIONED BY (pickup_date)
AS
SELECT 
    date(tpep_pickup_datetime) as pickup_date,
    hour(tpep_pickup_datetime) as pickup_hour,
    PULocationID as pickup_location_id,
    COUNT(*) as trip_count,
    AVG(trip_distance) as avg_distance,
    AVG(fare_amount) as avg_fare,
    AVG(tip_amount) as avg_tip,
    SUM(total_amount) as total_revenue
FROM demo.nyc_taxi.yellow_trips_raw
WHERE tpep_pickup_datetime IS NOT NULL
    AND trip_distance > 0
    AND fare_amount > 0
GROUP BY date(tpep_pickup_datetime), hour(tpep_pickup_datetime), PULocationID

### 日別集計テーブル（daily_stats）

In [ ]:
%%sql
CREATE OR REPLACE TABLE demo.nyc_taxi.daily_stats
USING iceberg
AS
SELECT 
    date(tpep_pickup_datetime) as pickup_date,
    dayofweek(date(tpep_pickup_datetime)) as day_of_week,
    CASE 
        WHEN dayofweek(date(tpep_pickup_datetime)) IN (1, 7) THEN 'Weekend'
        ELSE 'Weekday'
    END as day_type,
    COUNT(*) as trip_count,
    AVG(trip_distance) as avg_distance,
    AVG(fare_amount) as avg_fare,
    AVG(CASE WHEN trip_distance > 0 THEN fare_amount / trip_distance END) as avg_fare_per_mile,
    SUM(total_amount) as total_revenue
FROM demo.nyc_taxi.yellow_trips_raw
WHERE tpep_pickup_datetime IS NOT NULL
    AND trip_distance > 0
    AND fare_amount > 0
GROUP BY date(tpep_pickup_datetime)

In [ ]:
%%sql
CREATE OR REPLACE TABLE demo.nyc_taxi.daily_stats
USING iceberg
AS
SELECT 
    date(tpep_pickup_datetime) as pickup_date,
    dayofweek(date(tpep_pickup_datetime)) as day_of_week,
    CASE 
        WHEN dayofweek(date(tpep_pickup_datetime)) IN (1, 7) THEN 'Weekend'
        ELSE 'Weekday'
    END as day_type,
    COUNT(*) as trip_count,
    AVG(trip_distance) as avg_distance,
    AVG(fare_amount) as avg_fare,
    AVG(CASE WHEN trip_distance > 0 THEN fare_amount / trip_distance END) as avg_fare_per_mile,
    SUM(total_amount) as total_revenue
FROM demo.nyc_taxi.yellow_trips_raw
WHERE tpep_pickup_datetime IS NOT NULL
    AND trip_distance > 0
    AND fare_amount > 0
GROUP BY date(tpep_pickup_datetime)

## 5. 分析クエリの実行

In [ ]:
%%sql
SELECT 
    pickup_hour,
    SUM(trip_count) as total_trips,
    AVG(avg_fare) as average_fare
FROM demo.nyc_taxi.hourly_stats
GROUP BY pickup_hour
ORDER BY total_trips DESC

In [ ]:
%%sql
-- 平日と週末の比較
SELECT 
    day_type,
    SUM(trip_count) as total_trips,
    AVG(avg_distance) as avg_trip_distance,
    AVG(avg_fare) as avg_fare
FROM nyc_taxi.daily_stats
GROUP BY day_type

In [ ]:
%%sql
SELECT 
    day_type,
    SUM(trip_count) as total_trips,
    AVG(avg_distance) as avg_trip_distance,
    AVG(avg_fare) as avg_fare
FROM demo.nyc_taxi.daily_stats
GROUP BY day_type

## 6. Apache Supersetでの可視化

作成したIcebergテーブルは、Apache Supersetから可視化できます。

### Supersetへのアクセス
- URL: http://localhost:8088
- ユーザー名: admin
- パスワード: admin

### Trinoデータソースの設定手順
1. 画面右上の「Settings」 → 「Database Connections」をクリック
2. 画面右上の「Database」をクリック
3. 「Supported databases」のプルダウンから「Trino」を選択
4. 「SQLAlchemy URI」に以下の接続情報を入力：
   - `trino://trino@trino:8085/iceberg`
5. 「Test Connection」で接続を確認後、「Connect」をクリック

接続が完了したら、作成した集計テーブル（`nyc_taxi.hourly_stats`、`nyc_taxi.daily_stats`）をデータセットとして登録します。これにより、ドラッグ＆ドロップで様々なチャートを作成できるようになります。  
  
1. 画面右上の「+」メニュー → 「Data」→「Create Dataset」をクリック
2. Database に「Trino」を選択
3. Schema に「nyc_taxi」を選択
   - ここで、`nyc_taxi` は Iceberg テーブルを保存したスキーマ名です
4. Table に「yellow_trips_raw」を選択
5. 「CREATE DATASET AND CREATE CHART」をクリック